In [1]:
# Download drug review dataset from UCI
! wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"

--2023-06-04 21:57:18--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42989872 (41M) [application/x-httpd-php]
Saving to: 'drugsCom_raw.zip'

     0K .......... .......... .......... .......... ..........  0%  524K 80s
    50K .......... .......... .......... .......... ..........  0% 1.09M 59s
   100K .......... .......... .......... .......... ..........  0% 5.08M 42s
   150K .......... .......... .......... .......... ..........  0% 1.45M 38s
   200K .......... .......... .......... .......... ..........  0% 4.87M 32s
   250K .......... .......... .......... .......... ..........  0% 7.47M 28s
   300K .......... .......... .......... .......... ..........  0% 1.44M 28s
   350K .......... .......... .......... .......... ..........  0% 7

In [2]:
! unzip drugsCom_raw.zip

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


In [3]:
from datasets import load_dataset

In [5]:
data_files = {"train": "../data/drugsComTrain_raw.tsv", "test": "../data/drugsComTest_raw.tsv"}
dataset = load_dataset("csv", data_files=data_files, delimiter="\t") # delimiter is tab

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/Raj/.cache/huggingface/datasets/csv/default-643371abeec17c0c/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
drug_sample = dataset["train"].shuffle(seed=42).select(range(1000))
drug_sample[:5]

Loading cached shuffled indices for dataset at C:\Users\Raj\.cache\huggingface\datasets\csv\default-643371abeec17c0c\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-5df81ac101dbcc51.arrow


{'Unnamed: 0': [87571, 178045, 80482, 159268, 205477],
 'drugName': ['Naproxen', 'Duloxetine', 'Mobic', 'TriNessa', 'Pristiq'],
 'condition': ['Gout, Acute',
  'ibromyalgia',
  'Inflammatory Conditions',
  'Birth Control',
  'Depression'],
 'review': ['"like the previous person mention, I&#039;m a strong believer of aleve, it works faster for my gout than the prescription meds I take. No more going to the doctor for refills.....Aleve works!"',
  '"I have taken Cymbalta for about a year and a half for fibromyalgia pain. It is great\r\nas a pain reducer and an anti-depressant, however, the side effects outweighed \r\nany benefit I got from it. I had trouble with restlessness, being tired constantly,\r\ndizziness, dry mouth, numbness and tingling in my feet, and horrible sweating. I am\r\nbeing weaned off of it now. Went from 60 mg to 30mg and now to 15 mg. I will be\r\noff completely in about a week. The fibro pain is coming back, but I would rather deal with it than the side effects."',

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

In [13]:
dataset.keys()

dict_keys(['train', 'test'])

In [14]:
len(dataset["train"])

161297

In [16]:
len(dataset["train"].unique("Unnamed: 0"))

161297

In [19]:
print(f"Drugset: ", {len(dataset["train"].unique("drugName"))})
print(f"Conditionset: ", {len(dataset["train"].unique("condition"))})

Drugset:  {3436}
Conditionset:  {885}


In [17]:
# Test if the unnamed: 0 column is a unique patient ID column
for split in dataset.keys():
    assert len(dataset[split].unique("Unnamed: 0")) == len(dataset[split])

In [18]:
# Now that we know the unnamed: 0 column is a unique patient ID column, we can rename it to patient_id
dataset = dataset.rename_column("Unnamed: 0", "patient_id")
dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

In [20]:
# Normalize the conditions to lower case
dataset = dataset.map(lambda example: {"condition": example["condition"].lower()})

Map:   0%|          | 0/161297 [00:00<?, ? examples/s]

AttributeError: 'NoneType' object has no attribute 'lower'

In [21]:
# To remove the empty conditions, we filter out the rows where the condition is empty
dataset = dataset.filter(lambda example: example["condition"] is not None)

Filter:   0%|          | 0/161297 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53766 [00:00<?, ? examples/s]

In [25]:
# normalize the condition to lower case
dataset = dataset.map(lambda example: {"condition": example["condition"].lower()})

Map:   0%|          | 0/160398 [00:00<?, ? examples/s]

Map:   0%|          | 0/53471 [00:00<?, ? examples/s]

In [24]:
# before the condition mapping
dataset["train"][:5]

{'patient_id': [206461, 95260, 92703, 138000, 35696],
 'drugName': ['Valsartan',
  'Guanfacine',
  'Lybrel',
  'Ortho Evra',
  'Buprenorphine / naloxone'],
 'condition': ['Left Ventricular Dysfunction',
  'ADHD',
  'Birth Control',
  'Birth Control',
  'Opiate Dependence'],
 'review': ['"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
  '"My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. 

In [26]:
# after the condition mapping
dataset["train"][:5]

{'patient_id': [206461, 95260, 92703, 138000, 35696],
 'drugName': ['Valsartan',
  'Guanfacine',
  'Lybrel',
  'Ortho Evra',
  'Buprenorphine / naloxone'],
 'condition': ['left ventricular dysfunction',
  'adhd',
  'birth control',
  'birth control',
  'opiate dependence'],
 'review': ['"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
  '"My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. 

In [28]:
# compute review length and add it to the dataset
dataset = dataset.map(lambda example: {"review_length": len(example["review"].split())})
dataset["train"][:5]

Map:   0%|          | 0/160398 [00:00<?, ? examples/s]

Map:   0%|          | 0/53471 [00:00<?, ? examples/s]

{'patient_id': [206461, 95260, 92703, 138000, 35696],
 'drugName': ['Valsartan',
  'Guanfacine',
  'Lybrel',
  'Ortho Evra',
  'Buprenorphine / naloxone'],
 'condition': ['left ventricular dysfunction',
  'adhd',
  'birth control',
  'birth control',
  'opiate dependence'],
 'review': ['"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
  '"My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. 

In [29]:
# use the filter function to remove the reviews that are shorter than 30 words
dataset = dataset.filter(lambda example: example["review_length"] >= 30)
print(f"Number of examples after filtering: {dataset.num_rows}")

Filter:   0%|          | 0/160398 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53471 [00:00<?, ? examples/s]

Number of examples after filtering: {'train': 139529, 'test': 46434}


In [30]:
# unescape the html characters
import html

text = "I&#039;m on my second month and I&#039;m feeling great!!"
print(html.unescape(text))

I'm on my second month and I'm feeling great!!


In [31]:
# unescape the html characters
test_dataset = dataset.map(lambda example: {"review": html.unescape(example["review"])})

Map:   0%|          | 0/139529 [00:00<?, ? examples/s]

Map:   0%|          | 0/46434 [00:00<?, ? examples/s]

In [32]:
# use the map function's batched=True argument to apply the html.unescape function to all the examples in the dataset and do it faster
dataset = dataset.map(lambda example: {"review": html.unescape(example["review"])}, batched=True)

Map:   0%|          | 0/139529 [00:00<?, ? examples/s]

Map:   0%|          | 0/46434 [00:00<?, ? examples/s]

In [33]:
# setting up fast tokenizer functionality
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# tokenize the reviews
def tokenize_function(examples):
    return tokenizer(examples["review"], truncation=True)

In [34]:
# measure the time it takes to tokenize the dataset
import time
%time tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/139529 [00:00<?, ? examples/s]

Map:   0%|          | 0/46434 [00:00<?, ? examples/s]

CPU times: total: 2min 2s
Wall time: 15.8 s


In [35]:
# measure the time it takes to tokenize the dataset
import time
%time tokenized_dataset = dataset.map(tokenize_function, batched=False)

Map:   0%|          | 0/139529 [00:00<?, ? examples/s]

Map:   0%|          | 0/46434 [00:00<?, ? examples/s]

CPU times: total: 1min 1s
Wall time: 1min 1s


In [36]:
# setting up slow tokenizer functionality
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=False)

# tokenize the reviews
def tokenize_function(examples):
    return tokenizer(examples["review"], truncation=True)


In [37]:
%time tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/139529 [00:00<?, ? examples/s]

Map:   0%|          | 0/46434 [00:00<?, ? examples/s]

CPU times: total: 2min 9s
Wall time: 2min 9s


In [38]:
%time tokenized_dataset = dataset.map(tokenize_function, batched=False)

Map:   0%|          | 0/139529 [00:00<?, ? examples/s]

Map:   0%|          | 0/46434 [00:00<?, ? examples/s]

CPU times: total: 2min 31s
Wall time: 2min 30s


In [47]:
# Enable multiprocessing and use num_proc to specify the number of processes to use in the call to Dataset.map()
slow_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=False)

def slow_tokenize_function(examples):
    return slow_tokenizer(examples["review"], truncation=True)


In [50]:
tokenized_dataset = dataset.map(slow_tokenize_function, batched=True, num_proc=2)

Map (num_proc=2):   0%|          | 0/139529 [00:00<?, ? examples/s]

NameError: name 'slow_tokenizer' is not defined

In [61]:
# Enable multiprocessing and use num_proc to specify the number of processes to use in the call to Dataset.map()
fast_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=True, use_thread=True)

tokenized_dataset = dataset.map(lambda examples: fast_tokenizer(examples["review"]), batched=True)

Map:   0%|          | 0/139529 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (561 > 512). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: 

In [53]:
slow_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=False)


def slow_tokenize_function(examples):
    return slow_tokenizer(examples["review"], truncation=True)


tokenized_dataset = dataset.map(slow_tokenize_function, batched=True, num_proc=8)

Map (num_proc=8):   0%|          | 0/139529 [00:00<?, ? examples/s]

NameError: name 'slow_tokenizer' is not defined

In [62]:
# return all chunks of the text instead of just the first one
def tokenize_and_split(examples):
    return tokenizer(
        examples["review"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )

In [74]:
len(dataset["train"][0]["review"].split())

141

In [80]:
# let's test this on one example before using Dataset.map() on the whole dataset
result = tokenize_and_split(dataset["train"][0])
result.keys()
result
len(result["input_ids"])   
len(result["overflowing_tokens"]) 
# {len(chunk) for chunk in result}

47

In [81]:
tokenized_dataset = dataset.map(tokenize_and_split, batched=True)

Loading cached processed dataset at C:\Users\Raj\.cache\huggingface\datasets\csv\default-643371abeec17c0c\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-6684a81ae8cffbfb.arrow
Loading cached processed dataset at C:\Users\Raj\.cache\huggingface\datasets\csv\default-643371abeec17c0c\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-3f428df218baf063.arrow


In [82]:
len(tokenized_dataset["train"]), len(dataset["train"])

(139529, 139529)

In [85]:
# Another way to deal with the mismatched lengths is to repeat the values of each example as many times as it generates new features
def tokenize_and_split(examples):
    result = tokenizer(
        examples["review"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )
    # Extract mapping between new and old indices
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, value in examples.items():
        result[key] = [value[old_index] for old_index in sample_map]
    return result

In [86]:
tokenized_dataset = dataset.map(tokenize_and_split, batched=True)
tokenized_dataset

Map:   0%|          | 0/139529 [00:00<?, ? examples/s]

KeyError: 'overflow_to_sample_mapping'

In [87]:
# dataset.format() method to format the dataset in a way that is compatible with Pandas, PyTorch and TensorFlow
dataset.set_format("pandas")
dataset["train"][:3]

,patient_id,drugName,condition,review,rating,date,usefulCount,review_length
0,95260,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,141
1,92703,Lybrel,birth control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,134
2,138000,Ortho Evra,birth control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,89
